**LINK PARA BASE DE DADOS:**  (http://dados.recife.pe.gov.br/dataset/rede-de-educacao-municipal/resource/7c613836-9edd-4c0f-bc72-495008dd29c3)


Dados sobre as escolas municipais do recife

In [ ]:
!wget -q !wget -q https://dlcdn.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!tar xf spark-3.4.1-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql.functions import count, col, when
from pyspark.sql import functions as F

In [ ]:
# Iniciar uma sessão local e importar dados
from pyspark.sql import SparkSession
sc = SparkSession.builder.master('local[*]').getOrCreate()

In [ ]:
df = sc.read.csv("/content/info_unidadesensino_07102021.csv", header=True, inferSchema = True, sep = ";")

In [ ]:
df1 = df

In [ ]:
df1.dtypes

[('rpa', 'int'),
 ('tipo_cod', 'int'),
 ('tipo', 'string'),
 ('cod_escola', 'int'),
 ('escola', 'string'),
 ('inep', 'int'),
 ('rua', 'string'),
 ('numero', 'string'),
 ('cod_bairro', 'int'),
 ('bairro', 'string'),
 ('metragem', 'string'),
 ('qtd_alunos', 'int'),
 ('qtd_turmas', 'int'),
 ('qtd_professores', 'int'),
 ('escola_climatizada', 'string'),
 ('data_visita', 'string'),
 ('quadra_coberta', 'string'),
 ('quadra_descoberta', 'string'),
 ('biblioteca', 'string'),
 ('sala_recurso', 'string'),
 ('gestor', 'string'),
 ('longitude', 'double'),
 ('latitude', 'double')]

**VISUALIZAÇÃO DOS DADOS**

In [ ]:
df1.show()

+---+--------+----------------+----------+--------------------+--------+--------------------+------+----------+------------------+--------+----------+----------+---------------+------------------+-----------+--------------+-----------------+----------+------------+--------------------+---------+--------+
|rpa|tipo_cod|            tipo|cod_escola|              escola|    inep|                 rua|numero|cod_bairro|            bairro|metragem|qtd_alunos|qtd_turmas|qtd_professores|escola_climatizada|data_visita|quadra_coberta|quadra_descoberta|biblioteca|sala_recurso|              gestor|longitude|latitude|
+---+--------+----------------+----------+--------------------+--------+--------------------+------+----------+------------------+--------+----------+----------+---------------+------------------+-----------+--------------+-----------------+----------+------------+--------------------+---------+--------+
|  1|       1|ESCOLA MUNICIPAL|        90|ALMIRANTE SOARES ...|26120658|          

In [ ]:
for columns in df1: # um for para percorrer cada valor distinto de cada tabela e mostrar
  print(df1.select(columns).distinct().show())

+---+
|rpa|
+---+
|  1|
|  6|
|  3|
|  5|
|  4|
|  2|
+---+

None
+--------+
|tipo_cod|
+--------+
|       1|
|       5|
|       8|
|       2|
+--------+

None
+-----------------+
|             tipo|
+-----------------+
|CRECHE ESC.RECIFE|
|             CMEI|
| ESCOLA MUNICIPAL|
| CRECHE MUNICIPAL|
+-----------------+

None
+----------+
|cod_escola|
+----------+
|       148|
|       496|
|       516|
|        31|
|       251|
|        85|
|       451|
|       137|
|       458|
|       255|
|       481|
|       133|
|        78|
|       321|
|       974|
|       375|
|       950|
|       108|
|        34|
|       530|
+----------+
only showing top 20 rows

None
+--------------------+
|              escola|
+--------------------+
|DIACONO ABEL GUEIROS|
|            DO PILAR|
|           ESPERANCA|
|PROFESSOR ORLANDO...|
|ALTO JARDIM PROGR...|
|   MAEZINHA DO COQUE|
|      IBURA DE BAIXO|
|PROFESSOR MOACYR ...|
|ARTISTA PLASTICO ...|
|           DO JORDAO|
|     MONTEIRO LOBATO|
|     LAG

**TRANSFORMAÇÃO DOS DADOS**

In [ ]:
df1 = df1.withColumn("data_visita", when(col("data_visita").isNull(), "NAN").otherwise(col("data_visita")))  #Alterando os valores nulos por NAN só para ver se pegava

# df1 = df1.withColumn("data_visita", when(col("data_visita") == "valor", "valor alterad").otherwise(col("data_visita"))) # como alterar um valor diferente de null

In [ ]:
df1 = df1.withColumn("tipo", when(col("tipo") == "centros munic de ed.infantil", "CMEI").otherwise(col("tipo")))

In [ ]:
df1 = df1.drop("longitude", "latitude", "quadra_descoberta", "metragem", "tipo_cod", "inep") # dropando colunas que não irei usar

In [ ]:
df1 = df1.withColumnRenamed("rpa", "região politico administrativa") # renomeando coluna que não irei usar

In [ ]:
df1.show()

+------------------------------+----------------+----------+--------------------+--------------------+------+----------+------------------+----------+----------+---------------+------------------+-----------+--------------+----------+------------+--------------------+
|região politico administrativa|            tipo|cod_escola|              escola|                 rua|numero|cod_bairro|            bairro|qtd_alunos|qtd_turmas|qtd_professores|escola_climatizada|data_visita|quadra_coberta|biblioteca|sala_recurso|              gestor|
+------------------------------+----------------+----------+--------------------+--------------------+------+----------+------------------+----------+----------+---------------+------------------+-----------+--------------+----------+------------+--------------------+
|                             1|ESCOLA MUNICIPAL|        90|ALMIRANTE SOARES ...|           CAMUTANGA|   180|        51|           CABANGA|       387|        16|             12|               S

In [ ]:
df1 = df1.drop("data_visita") # deletando mais uma colunas

In [ ]:
biblioteca = df1.groupBy('biblioteca').agg(F.count('biblioteca')).orderBy(F.count('biblioteca').desc()).show(10)

+----------+-----------------+
|biblioteca|count(biblioteca)|
+----------+-----------------+
|       SIM|              180|
|       NÃO|              171|
+----------+-----------------+



In [ ]:
df1.select("escola").distinct().count()

351

In [ ]:
bairro = df1.select("bairro").agg(F.count('bairro')).show()

+-------------+
|count(bairro)|
+-------------+
|          351|
+-------------+



In [ ]:
df1.select("numero").distinct().show(20)

+------+
|numero|
+------+
|    07|
|   125|
|  1265|
|   591|
|    51|
|  3895|
|   169|
|   470|
|    54|
|   700|
|   155|
|   200|
|   279|
|   685|
|   415|
|    69|
|   419|
|  1470|
|   112|
|    42|
+------+
only showing top 20 rows



In [ ]:
numero = df1.groupBy('numero').agg(F.count('numero')).orderBy(F.count('numero').desc()).show()

+------+-------------+
|numero|count(numero)|
+------+-------------+
|   S/N|           77|
|    SN|            9|
|   395|            5|
|   200|            4|
|    35|            4|
|    30|            3|
|    52|            3|
|   195|            3|
|   320|            3|
|   300|            3|
|   100|            3|
|    41|            3|
|   111|            3|
|   115|            3|
|   134|            3|
|   127|            3|
|    10|            3|
|    28|            2|
|    85|            2|
|   356|            2|
+------+-------------+
only showing top 20 rows



In [ ]:
df1 = df1.withColumn("numero", when(col("numero") == "S/N", "SN").otherwise(col("numero")))

In [ ]:
numero = df1.groupBy('numero').agg(F.count('numero')).orderBy(F.count('numero').desc()).show(10)

+------+-------------+
|numero|count(numero)|
+------+-------------+
|    SN|           86|
|   395|            5|
|   200|            4|
|    35|            4|
|   100|            3|
|    52|            3|
|    41|            3|
|   115|            3|
|   134|            3|
|   195|            3|
+------+-------------+
only showing top 10 rows



**CONSULTAS**

In [ ]:
df1.show()

+------------------------------+----------------+----------+--------------------+--------------------+------+----------+------------------+----------+----------+---------------+------------------+--------------+----------+------------+--------------------+
|região politico administrativa|            tipo|cod_escola|              escola|                 rua|numero|cod_bairro|            bairro|qtd_alunos|qtd_turmas|qtd_professores|escola_climatizada|quadra_coberta|biblioteca|sala_recurso|              gestor|
+------------------------------+----------------+----------+--------------------+--------------------+------+----------+------------------+----------+----------+---------------+------------------+--------------+----------+------------+--------------------+
|                             1|ESCOLA MUNICIPAL|        90|ALMIRANTE SOARES ...|           CAMUTANGA|   180|        51|           CABANGA|       387|        16|             12|               SIM|           NÃO|       SIM|       

Mostrar escolas em santo amaro sem climatização

In [ ]:
df1.filter((df1['bairro'] == 'SANTO AMARO') & (df1['escola_climatizada'].isin(['NÂO']))).show()

+------------------------------+----------------+----------+--------------------+--------------------+------+----------+-----------+----------+----------+---------------+------------------+--------------+----------+------------+--------------------+
|região politico administrativa|            tipo|cod_escola|              escola|                 rua|numero|cod_bairro|     bairro|qtd_alunos|qtd_turmas|qtd_professores|escola_climatizada|quadra_coberta|biblioteca|sala_recurso|              gestor|
+------------------------------+----------------+----------+--------------------+--------------------+------+----------+-----------+----------+----------+---------------+------------------+--------------+----------+------------+--------------------+
|                             1|ESCOLA MUNICIPAL|       289|CIDADAO HERBERT D...|     ARNOBIO MARQUES|   310|       108|SANTO AMARO|       207|        12|             12|               NÂO|           NÃO|       NÃO|         SIM|ANA CLAUDIA DO NA...|


Mostrar escolas na Boa Vista com climatização

In [ ]:
df1.filter((df1['bairro'] == 'BOA VISTA') & (df1['escola_climatizada'].isin(['SIM']))).show()

+------------------------------+----------------+----------+--------------------+------------------+------+----------+---------+----------+----------+---------------+------------------+--------------+----------+------------+--------------------+
|região politico administrativa|            tipo|cod_escola|              escola|               rua|numero|cod_bairro|   bairro|qtd_alunos|qtd_turmas|qtd_professores|escola_climatizada|quadra_coberta|biblioteca|sala_recurso|              gestor|
+------------------------------+----------------+----------+--------------------+------------------+------+----------+---------+----------+----------+---------------+------------------+--------------+----------+------------+--------------------+
|                             1|ESCOLA MUNICIPAL|        66|       PEDRO AUGUSTO|BARAO DE SAO BORJA|   279|        86|BOA VISTA|       387|        15|             13|               SIM|           SIM|       SIM|         SIM|JOSENIDE MATIAS P...|
|               

Mostrar escolas de Recife que não possuem biblioteca

In [ ]:
df1.filter((df1['bairro'] == 'RECIFE') & (df1['biblioteca'].isin(['NÃO']))).show()

+------------------------------+-----------------+----------+--------------------+---------------+------+----------+------+----------+----------+---------------+------------------+--------------+----------+------------+--------------------+
|região politico administrativa|             tipo|cod_escola|              escola|            rua|numero|cod_bairro|bairro|qtd_alunos|qtd_turmas|qtd_professores|escola_climatizada|quadra_coberta|biblioteca|sala_recurso|              gestor|
+------------------------------+-----------------+----------+--------------------+---------------+------+----------+------+----------+----------+---------------+------------------+--------------+----------+------------+--------------------+
|                             1|             CMEI|       425|DOUTOR ALBERICO D...|  CAIS DO APOLO|   925|        19|RECIFE|       103|         7|              6|               NÂO|           NÃO|       NÃO|         NÃO|ANA CRISTINA TEIX...|
|                             1|CREC

Trazendo as escolas da região 6 que possuem mais de 500 alunos e não possue uma biblioteca na escola

In [ ]:
df1.filter((df1['região politico administrativa'] == 6) & (df1['qtd_alunos'] > 500) & (df1['biblioteca'] == "NÃO")).show()

+------------------------------+----------------+----------+--------------------+-----------+------+----------+------+----------+----------+---------------+------------------+--------------+----------+------------+--------------------+
|região politico administrativa|            tipo|cod_escola|              escola|        rua|numero|cod_bairro|bairro|qtd_alunos|qtd_turmas|qtd_professores|escola_climatizada|quadra_coberta|biblioteca|sala_recurso|              gestor|
+------------------------------+----------------+----------+--------------------+-----------+------+----------+------+----------+----------+---------------+------------------+--------------+----------+------------+--------------------+
|                             6|ESCOLA MUNICIPAL|       164|POETA PAULO BANDE...|DAS PANELAS|    28|       884| COHAB|       541|        24|             17|               SIM|           NÃO|       NÃO|         SIM|MARIA ANDREZA CAB...|
|                             6|ESCOLA MUNICIPAL|       

Trazendo as escolas do bairro de santo amaro que não possuem recurso de sala

In [ ]:
df1.filter((df1['bairro'] == "SANTO AMARO" ) & (df1['sala_recurso'] == 'NÃO')).show()

+------------------------------+----------------+----------+--------------------+---------+------+----------+---------+----------+----------+---------------+------------------+--------------+----------+------------+------+
|região politico administrativa|            tipo|cod_escola|              escola|      rua|numero|cod_bairro|   bairro|qtd_alunos|qtd_turmas|qtd_professores|escola_climatizada|quadra_coberta|biblioteca|sala_recurso|gestor|
+------------------------------+----------------+----------+--------------------+---------+------+----------+---------+----------+----------+---------------+------------------+--------------+----------+------------+------+
|                             3|ESCOLA MUNICIPAL|       405|PADRE ANTONIO HEN...|PAISSANDU|   112|        86|BOA VISTA|       115|         8|              4|               SIM|           NÃO|       SIM|         NÃO|  null|
+------------------------------+----------------+----------+--------------------+---------+------+----------

Trazendo os valores a quantidade das escolas das regiões que possuem ou não recurso de sala de aula

In [ ]:
df1.groupBy('região politico administrativa', 'sala_recurso').count().orderBy("região politico administrativa").show()

+------------------------------+------------+-----+
|região politico administrativa|sala_recurso|count|
+------------------------------+------------+-----+
|                             1|         NÃO|   10|
|                             1|         SIM|   16|
|                             2|         SIM|   21|
|                             2|         NÃO|   28|
|                             3|         SIM|   31|
|                             3|         NÃO|   47|
|                             4|         NÃO|   35|
|                             4|         SIM|   24|
|                             5|         NÃO|   25|
|                             5|         SIM|   27|
|                             6|         NÃO|   53|
|                             6|         SIM|   34|
+------------------------------+------------+-----+



Procurando uma escola que tem mais turmas que a quantidade de alunos e encontramos apenas uma escola que nao tem alunos e nem professores, acredito que esta escola não está em atividade na época da database

In [ ]:
df1.filter((df1['qtd_turmas']) > (df1['qtd_alunos'])).show()

+------------------------------+----------------+----------+--------------------+----------+------+----------+--------+----------+----------+---------------+------------------+--------------+----------+------------+------+
|região politico administrativa|            tipo|cod_escola|              escola|       rua|numero|cod_bairro|  bairro|qtd_alunos|qtd_turmas|qtd_professores|escola_climatizada|quadra_coberta|biblioteca|sala_recurso|gestor|
+------------------------------+----------------+----------+--------------------+----------+------+----------+--------+----------+----------+---------------+------------------+--------------+----------+------------+------+
|                             3|ESCOLA MUNICIPAL|       423|PROFESSOR NILO PE...|DO ARRAIAL|  4744|       485|MONTEIRO|         0|        12|              0|               NÂO|           SIM|       SIM|         SIM|  null|
+------------------------------+----------------+----------+--------------------+----------+------+---------

Essa consulta mostra asescolas que possuem menos professores que a quantidade de turmas

In [ ]:
teste = df1.filter((df1['qtd_professores']) < (df1['qtd_turmas'])).show()

+------------------------------+-----------------+----------+--------------------+--------------------+------+----------+------------------+----------+----------+---------------+------------------+--------------+----------+------------+--------------------+
|região politico administrativa|             tipo|cod_escola|              escola|                 rua|numero|cod_bairro|            bairro|qtd_alunos|qtd_turmas|qtd_professores|escola_climatizada|quadra_coberta|biblioteca|sala_recurso|              gestor|
+------------------------------+-----------------+----------+--------------------+--------------------+------+----------+------------------+----------+----------+---------------+------------------+--------------+----------+------------+--------------------+
|                             1| ESCOLA MUNICIPAL|        90|ALMIRANTE SOARES ...|           CAMUTANGA|   180|        51|           CABANGA|       387|        16|             12|               SIM|           NÃO|       SIM|   